<a href="https://colab.research.google.com/github/arturbernardo/data_stars/blob/main/Gaia2Exploration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyvo
!pip install mocpy
!pip install astropy
!pip install ipyaladin


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 997.6/997.6 kB 9.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 28.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 220.7/220.7 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 765.5/765.5 kB 26.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 40.4 MB/s eta 0:00:00


In [2]:
# Access astronomical databases
from pyvo import registry  # version >=1.6

# Moc and HEALPix tools
from mocpy import MOC

# Coordinates manipulation
from astropy.coordinates import SkyCoord

# Sky visualization
from ipyaladin import Aladin  # version >=0.4.0

# For plots
import matplotlib.pyplot as plt


In [13]:
# the catalogue name in VizieR
# CATALOGUE = "J/A+A/616/A10"
CATALOGUE = "J/A+A/620/A128"


In [14]:
# each resource in the VO has an identifier, called ivoid. For vizier catalogs,
# the VO ids can be constructed like this:
catalogue_ivoid = f"ivo://CDS.VizieR/{CATALOGUE}"
# the actual query to the registry
voresource = registry.search(ivoid=catalogue_ivoid)[0]

In [15]:
# We can print metadata information about the catalogue
voresource.describe(verbose=True)

Gaia DR2 study of Herbig Ae/Be stars
Short Name: J/A+A/620/A128
IVOA Identifier: ivo://cds.vizier/j/a+a/620/a128
Access modes: conesearch, tap#aux, web
- tap#aux: http://tapvizier.cds.unistra.fr/TAPVizieR/tap
- webpage: http://vizier.cds.unistra.fr/viz-bin/VizieR-2?-source=J/A+A/620/A128
- conesearch: https://vizier.cds.unistra.fr/viz-
 bin/conesearch/J/A+A/620/A128/hqsample?, description: Cone search capability
 for table J/A+A/620/A128/hqsample (Main parameters of each Herbig Ae/Be star
 belonging to the high quality 218 sources (table 1))
- conesearch: https://vizier.cds.unistra.fr/viz-
 bin/conesearch/J/A+A/620/A128/lqsample?, description: Cone search capability
 for table J/A+A/620/A128/lqsample (Main parameters of each Herbig Ae/Be star
 belonging to the low quality 34 sources (table 2))

We use Gaia Data Release 2 (DR2, Cat. I/345) to place 252 Herbig Ae/Be stars
in the HR diagram and investigate their characteristics and properties. For
all known Herbig Ae/Be stars with paralla

In [16]:
voresource.get_tables()

{'J/A+A/620/A128/hqsample': <VODataServiceTable name="J/A+A/620/A128/hqsample">... 51 columns ...</VODataServiceTable>,
 'J/A+A/620/A128/lqsample': <VODataServiceTable name="J/A+A/620/A128/lqsample">... 51 columns ...</VODataServiceTable>,
 'J/A+A/620/A128/refs': <VODataServiceTable name="J/A+A/620/A128/refs">... 5 columns ...</VODataServiceTable>}

In [17]:
tables = voresource.get_tables()
tables_names = list(tables.keys())
tables_names

['J/A+A/620/A128/hqsample', 'J/A+A/620/A128/lqsample', 'J/A+A/620/A128/refs']

In [18]:
# get the first table of the catalogue
first_table_name = tables_names[0]

# execute a synchronous ADQL query
tap_service = voresource.get_service("tap")
tap_records = tap_service.search(
    f'select TOP 10 * from "{first_table_name}"',
)
tap_records.fielddescs

[<FIELD ID="recno" datatype="int" name="recno" ucd="meta.record"/>,
 <FIELD ID="Name" arraysize="*" datatype="char" name="Name" ucd="meta.id;meta.main"/>,
 <FIELD ID="RAICRS" datatype="double" name="RAICRS" ref="coosys_c36200128hqsample" ucd="pos.eq.ra;meta.main" unit="deg"/>,
 <FIELD ID="DEICRS" datatype="double" name="DEICRS" ref="coosys_c36200128hqsample" ucd="pos.eq.dec;meta.main" unit="deg"/>,
 <FIELD ID="plx" datatype="double" name="plx" ucd="pos.parallax.trig" unit="mas"/>,
 <FIELD ID="e_plx" datatype="double" name="e_plx" ucd="stat.error;pos.parallax" unit="mas"/>,
 <FIELD ID="Dist" datatype="double" name="Dist" ucd="pos.distance;pos.heliocentric" unit="pc"/>,
 <FIELD ID="E_Dist" datatype="double" name="E_Dist" ucd="stat.error;stat.max" unit="pc"/>,
 <FIELD ID="e_Dist" datatype="double" name="e_Dist" ucd="stat.error" unit="pc"/>,
 <FIELD ID="Teff" datatype="int" name="Teff" ucd="phys.temperature.effective" unit="K"/>,
 <FIELD ID="E_Teff" datatype="short" name="E_Teff" ucd="stat

In [12]:
tap_records.infos

{'QUERY_STATUS': 'OK',
 'PROVIDER': 'CDS',
 'QUERY': 'select TOP 10 * from "J/A+A/616/A10/tablea1a"',
 'ivoid_service': 'ivo://CDS.VizieR/TAP',
 'publisher': 'ivo://CDS',
 'server_software': 'TAPVizieR-Vollt/1.1.3',
 'service_protocol': 'ivo://ivoa.net/std/TAP/v1.1',
 'request_date': '2025-04-06 16:07:24',
 'contact': 'cds-question@unistra.fr',
 'ivoid': 'ivo://CDS.VizieR/J/A+A/616/A10',
 'reference_url': 'https://cdsarc.cds.unistra.fr/viz-bin/cat/J/A+A/616/A10',
 'rights_uri': 'https://cds.unistra.fr/vizier-org/licences_vizier.html',
 'creator': 'Gaia Collaboration',
 'article': '2018A&A...616A..10G',
 'publication_date': '2018-08-10 12:38:01+02',
 'product_type': 'Critical Comp.'}

In [25]:
print(tap_records)

<DALResultsTable length=10>
recno    Name          RAICRS       ... SimbadName      _RA_icrs           _DE_icrs     
                        deg         ...                   deg                deg        
int32   object        float64       ...   object        float64            float64      
----- ---------- ------------------ ... ---------- ------------------ ------------------
    1     AB Aur           73.94125 ...     AB Aur           73.94125  30.55111111111111
    2     AK Sco 253.68666666666664 ...     AK Sco 253.68666666666664  -36.8886111111111
    3     AS 310  278.3383333333333 ...     AS 310  278.3383333333333 -4.968333333333333
    4     AS 470  324.0591666666666 ...     AS 470  324.0591666666666   57.3586111111111
    5     AS 477  328.1420833333333 ...     AS 477  328.1420833333333  47.22888888888888
    6  BD+30 549 52.332499999999996 ...  BD+30 549 52.332499999999996 31.415833333333328
    7 BD+41 3731 306.06541666666664 ... BD+41 3731 306.06541666666664  42.30027777

NameError: name 'pd' is not defined

ˊ
 **ndice** de cor=BP−RP
Este índice de cor, que pode ser positivo ou negativo, dá uma indicação do tipo de cor da estrela. Estrelas mais azuis têm um índice de cor negativo (porque o BP será menor do que o RP), enquanto estrelas mais vermelhas terão um índice de cor positivo (porque o RP será maior do que o BP).

Índice de cor negativo → Estrela azul.

Índice de cor positivo → Estrela vermelha.

Por exemplo:

Estrelas azuis: Têm um índice de cor BP - RP negativo, o que significa que elas têm uma magnitude maior na banda azul (BP) do que na banda vermelha (RP).

Estrelas vermelhas: Têm um índice de cor BP - RP positivo, o que indica que elas são mais brilhantes na banda vermelha.

phot_proc_mode : Photometry processing mode (byte)

This flag indicates the photometric calibration process used for the source. The process is determined by the availability of colour information derived from the internally calibrated mean BP and RP source spectra. The following values are defined for Gaia DR3:

• 0: this corresponds to the ‘gold’ photometric dataset. Sources in this dataset have complete colour information.
• 1: this corresponds to the ‘silver’ photometric dataset. Sources in this dataset have incomplete colour information and therefore were calibrated using an iterative process that estimated the missing colour information from the source mean G and either BP or RP photometry (depending on which band had full colour information available) using empirical relationships derived from the gold dataset.
• 2: this corresponds to the ‘bronze’ photometric dataset. Sources in this dataset had insufficient colour information and therefore were calibrated using default colour information derived from the gold dataset.

bp_rp : BP - RP colour (float, Magnitude[mag])



phot_g_mean_mag : G-band mean magnitude (float, Magnitude[mag])

Mean magnitude in the G band. This is computed from the G-band mean flux (phot_g_mean_flux ) applying the magnitude zero-point in the Vega scale (see Section 5.4.1).

No error is provided for this quantity as the error distribution is only symmetric in flux space. This converts to an asymmetric error distribution in magnitude space which cannot be represented by a single error value.

BY CHATGPT:

3. Exemplo de campos relacionados à cor no Gaia DR2:
Aqui estão alguns campos do Gaia DR2 que podem ser relevantes:

BP_mag: A magnitude da estrela na banda azul (BP).

RP_mag: A magnitude da estrela na banda vermelha (RP).

G_mag: A magnitude na banda G (visível).

BP-RP: O índice de cor que calcula a diferença entre as magnitudes BP e RP.